In [ ]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,1905.99,-0.01,0.03,0.00,0.00,-0.0,0.00
3,-0.01,610.40,-0.01,0.01,0.00,0.0,-0.00
4,0.03,-0.01,172.28,0.00,-0.00,0.0,-0.00
5,0.00,0.01,0.00,40.23,-0.00,-0.0,0.00
6,0.00,0.00,-0.00,-0.00,8.03,0.0,0.00
7,-0.00,0.00,0.00,-0.00,0.00,1.5,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.0,0.25


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00005,0.00000,0.00001,-0.00006,0.00001
3,-0.00001,1.00000,-0.00002,0.00003,0.00000,0.00008,-0.00005
4,0.00005,-0.00002,1.00000,0.00003,-0.00001,0.00003,-0.00001
5,0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00001,0.00003
6,0.00001,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00005
7,-0.00006,0.00008,0.00003,-0.00001,0.00002,1.00000,-0.00001
8,0.00001,-0.00005,-0.00001,0.00003,0.00005,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.043561946406049844

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.20728554e+05 5.17782194e+04 3.95753349e+03 2.02534220e+02
 7.20350769e+00 1.92191572e-01 1.03191601e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998138,-0.059063,-0.014804,-0.003431,-0.000684,-0.000130,-0.000021
1,0.057988,0.996301,-0.061974,-0.013148,-0.002614,-0.000486,-0.000082
2,0.018103,0.060048,0.996011,-0.062362,-0.011641,-0.002172,-0.000359
3,0.005239,0.016401,0.060404,0.995509,-0.069708,-0.012388,-0.002038
4,0.001381,0.004316,0.015314,0.067274,0.992786,-0.096776,-0.015143
5,0.000382,0.001184,0.004206,0.018038,0.091900,0.981837,-0.164943
6,0.000129,0.000401,0.001421,0.006084,0.030466,0.162703,0.986185


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00186167, 0.00369852, 0.00398939, 0.00449103, 0.00721394,
       0.01816318, 0.01381531])